In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import lightgbm as lgbm
from sklearn.base import clone
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold,GroupKFold,StratifiedKFold
from sklearn.metrics import roc_auc_score
# from bayes_opt import BayesianOptimization

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


/kaggle/input/widsdatathon2021/SampleSubmissionWiDS2021.csv
/kaggle/input/widsdatathon2021/SolutionTemplateWiDS2021.csv
/kaggle/input/widsdatathon2021/DataDictionaryWiDS2021.csv
/kaggle/input/widsdatathon2021/UnlabeledWiDS2021.csv
/kaggle/input/widsdatathon2021/TrainingWiDS2021.csv


In [4]:
train = pd.read_csv("../input/widsdatathon2021/TrainingWiDS2021.csv")

test = pd.read_csv("../input/widsdatathon2021/UnlabeledWiDS2021.csv")
del train["Unnamed: 0"],test["Unnamed: 0"]

In [5]:
cat_cols = train.select_dtypes(include=['object']).columns
train[cat_cols] = train[cat_cols].fillna("missing")
test[cat_cols] = test[cat_cols].fillna("missing")

from sklearn import preprocessing
for col in cat_cols:
    print(col)
    le = preprocessing.LabelEncoder()
    le.fit(pd.concat([train[col],test[col]],axis=0).astype("str"))
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])

ethnicity
gender
hospital_admit_source
icu_admit_source
icu_stay_type
icu_type


In [6]:
features = [col for col in train.columns if col not in ["diabetes_mellitus",
#                                                         "istrain",
                                                        "encounter_id"]]

print(len(features))
features[-10:]

178


['h1_arterial_po2_min',
 'h1_pao2fio2ratio_max',
 'h1_pao2fio2ratio_min',
 'aids',
 'cirrhosis',
 'hepatic_failure',
 'immunosuppression',
 'leukemia',
 'lymphoma',
 'solid_tumor_with_metastasis']

In [7]:

y = train["diabetes_mellitus"]
groups = train["hospital_id"]
gkf = GroupKFold(n_splits=10)
oof = np.zeros(len(train))
score_list = []
fold = 1
test_preds = []

for train_index, test_index in gkf.split(train,groups = groups):

    X_train, X_val = train.iloc[train_index], train.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    
    print(X_train.shape,X_val.shape)
    print(y_train.mean(),y_val.mean())
    
    y_pred_list = []
    for seed in [1]:
        dtrain = lgbm.Dataset(X_train[features], y_train,categorical_feature=["apache_2_diagnosis",
                                                                              "apache_3j_diagnosis"
])
        dvalid = lgbm.Dataset(X_val[features], y_val,categorical_feature=["apache_2_diagnosis", 
                                                                          "apache_3j_diagnosis"
])
        print(seed)
        params = {"objective": "binary",
              "metric": "auc",
              "verbosity": -1,
              "boosting_type": "gbdt",
              "feature_fraction":0.3,
              "num_leaves": 50,
              "lambda_l1":2,
              "lambda_l2":2,
              "learning_rate":0.08,
              'min_child_samples': 0,
                  "scale_pos_weight":1,
              "bagging_fraction":0.9,
              "bagging_freq":1}
        params["seed"] = seed
        model = lgbm.train(params,
                        dtrain,
                        valid_sets=[dtrain, dvalid],
                        verbose_eval=200,
                        num_boost_round=100000,
                        early_stopping_rounds=200
                    )
    
        y_pred_list.append(model.predict(X_val[features]))
        test_preds.append(model.predict(test[features]))
        
    
    oof[test_index] = np.mean(y_pred_list,axis=0)    
    score = roc_auc_score(y_val, oof[test_index])
    score_list.append(score)
    print(f"AUC Fold-{fold} : {score}")
    print("***********")
    fold+=1

np.mean(score_list)

(117140, 180) (13017, 180)
0.21097831654430596 0.2640393331796881
1


/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Datas

Training until validation scores don't improve for 200 rounds
[200]	training's auc: 0.919513	valid_1's auc: 0.867989
Early stopping, best iteration is:
[147]	training's auc: 0.908199	valid_1's auc: 0.868307
AUC Fold-1 : 0.8683073430912405
***********
(117143, 180) (13014, 180)
0.21900583048069452 0.1917934532042416
1


/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Datas

Training until validation scores don't improve for 200 rounds
[200]	training's auc: 0.918716	valid_1's auc: 0.857644
[400]	training's auc: 0.948103	valid_1's auc: 0.857723
Early stopping, best iteration is:
[339]	training's auc: 0.940545	valid_1's auc: 0.858508
AUC Fold-2 : 0.8585082776290706
***********
(117140, 180) (13017, 180)
0.2152296397473109 0.2257816701236844
1


...

/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Datas

Training until validation scores don't improve for 200 rounds
[200]	training's auc: 0.918951	valid_1's auc: 0.865487
[400]	training's auc: 0.94856	valid_1's auc: 0.866868
Early stopping, best iteration is:
[381]	training's auc: 0.946389	valid_1's auc: 0.866894
AUC Fold-9 : 0.8668941745224771
***********
(117144, 180) (13013, 180)
0.21545277607047736 0.22377622377622378
1


/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Datas

Training until validation scores don't improve for 200 rounds
[200]	training's auc: 0.919727	valid_1's auc: 0.866395
[400]	training's auc: 0.948724	valid_1's auc: 0.865097
Early stopping, best iteration is:
[217]	training's auc: 0.922961	valid_1's auc: 0.866702
AUC Fold-10 : 0.8667022482269735
***********


0.8612740331356562